# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.0 kB ? eta -:--:--
     -- ------------------------------------ 61.4/990.0 kB 3.2 MB/s eta 0:00:01
     --- ----------------------------------- 92.2/990.0 kB 1.3 MB/s eta 0:00:01
     --- ----------------------------------- 92.2/990.0 kB 1.3 MB/s eta 0:00:01
     ----- ------------------------------ 143.4/990.0 kB 774.0 kB/s eta 0:00:02
     ----- ------------------------------ 143.4/990.0 kB 774.0 kB/s eta 0:00:02
     ------- ---------------------------- 194.6/990.0 kB 692.9 kB/s eta 0:00:02
     ------- ---------------------------- 204.8/990.0 kB 655.1 kB/s eta 0:00:02
     -------- --------------------------- 235.5/990.0 kB 654.9 kB/s eta 0:00:02
     -------- --------------------------- 235.5/990.0 kB 654.9 kB/s eta 0:00:02
     ----------- ------------------------ 307.2/990.0 kB 678.0 kB/s eta 0:00:02
     ------------ ----------------------- 337.9/990.0 kB 655.4 kB/

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [7]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [24]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位经验丰富的生物学专家。 你对生物学领域有着深厚的了解，喜欢用平实的语言解答复杂的生物学问题。 当遇到超出你知识范围的问题时，你会诚实地表示不清楚答案。

这是一个问题： 
{input}"""

cs_template = """你是一位杰出的计算机科学家。 你对计算机科学领域的各种理论和实践有着深入的理解，并能用简明的方式解释艰涩的技术概念。 如果某个问题超出了你的专业领域，你会毫不犹豫地承认自己不了解。

这是一个问题： 
{input}"""

cn_template = """你是一位对汉语言文学有着深刻见解的学者。 你对中国的诗歌、小说、戏剧及其他文学形式充满热情，并能用简洁的方式阐述文学作品的深层含义。 遇到你不熟悉的文学作品或议题时，你会直言不讳地承认自己尚需研究。

这是一个问题：
{input}"""

In [25]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": cs_template,
    },
     {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": cn_template,
    },
]

In [26]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url="https://api.xiaoai.plus/v1")

In [27]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [28]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [29]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [30]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [31]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [32]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [33]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [34]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [35]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [37]:
print(chain.invoke("null point exception是什么？?"))



> Entering new MultiPromptChain chain...
计算机: {'input': 'null point exception是什么？?'}
> Finished chain.
{'input': 'null point exception是什么？?', 'text': '\n\n空指针异常（Null Point Exception）是一种在程序中发生的错误，通常是由于程序试图对一个空对象进行操作而引起的。空对象指的是没有被实例化或者被赋值为null的对象。当程序试图调用空对象的方法或者访问空对象的属性时，就会发生空指针异常。'}


In [16]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要确定大于40的质数。40不是质数，我们从41开始检查。41是质数，但41+1=42不能被3整除。接下来是43，43+1=44也不能被3整除。然后是47，47+1=48可以被3整除。所以大于40的第一个质数是47，47+1=48可以被3整除。'}


In [17]:
router_chain.verbose = True

In [36]:
print(chain.invoke("langchain是什么？"))



> Entering new MultiPromptChain chain...
None: {'input': 'langchain是什么？'}
> Finished chain.
{'input': 'langchain是什么？', 'history': '', 'text': ' langchain是一个区块链技术公司，专门致力于开发和运营基于区块链的多语言智能合约平台。该平台的目的是为了提供一种更加安全和高效的方式来执行智能合约，并且能够支持多种语言编写智能合约，使得开发者能够更加灵活地创建智能合约。\n'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains